In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Autism_spectrum_disorder_(ASD)"
cohort = "GSE113842"

# Input paths
in_trait_dir = "../../input/GEO/Autism_spectrum_disorder_(ASD)"
in_cohort_dir = "../../input/GEO/Autism_spectrum_disorder_(ASD)/GSE113842"

# Output paths
out_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/GSE113842.csv"
out_gene_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/gene_data/GSE113842.csv"
out_clinical_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/clinical_data/GSE113842.csv"
json_path = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Autism-like phenotype and risk gene-RNA deadenylation by CPEB4 mis-splicing"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['genotype: WT', 'genotype: HD', 'genotype: CPEB4 GT/+', 'genotype: CPEB4 KO', 'genotype: CTRL', 'genotype: TgCPEB4{delta}4'], 1: ['condition: CPEB1 RIP', 'condition: none', 'condition: CPEB4 RIP', 'condition: IgG RIP', 'group: INPUT', 'group: WASH', 'group: ELUTED'], 2: [nan, 'replicate: 1', 'replicate: 2', 'replicate: 3'], 3: [nan, 'tissue: Cortex/Striatum'], 4: [nan, 'age: 6 week-old'], 5: [nan, 'rna fraction: total RNA', 'rna fraction: Enriched in short poly(A)-tail RNA', 'rna fraction: Enriched in long poly(A)-tail RNA']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
from typing import Optional, Callable

# Define the sample characteristics dictionary from the previous output
sample_characteristics_dict = {
    0: ['group: CTRL', 'group: ASD'], 
    1: ['material: INPUT', 'material: WASH', 'material: ELUTED'], 
    2: ['age: 7', 'age: 9', 'age: 5', 'age: 13', 'age: 20/21', 'age: 11', 'age: 12', 'age: 21/23'], 
    3: ['postmorten interval (h): 16', 'postmorten interval (h): 12', 'postmorten interval (h): 21', 'postmorten interval (h): 3', 'postmorten interval (h): 28', 'postmorten interval (h): 11/19', 'postmorten interval (h): 27', 'postmorten interval (h): 22', 'postmorten interval (h): 14/18'], 
    4: ['hybridization batch: A', 'hybridization batch: B'], 
    5: ['tissue: Prefrontal Cortex BA9', 'tissue: Prefrontal Cortex BA8/9', 'tissue: Prefrontal Cortex BA8'], 
    6: ['rna fraction: total RNA', 'rna fraction: Enriched in short poly(A)-tail RNA', 'rna fraction: Enriched in long poly(A)-tail RNA']
}

# 1. Check Gene Expression Data Availability
# Based on series title and summary, it appears to be RNA expression data
is_gene_available = True  # RNA data from deadenylation by CPEB4 mis-splicing

# 2.1 Data Availability
trait_row = 0  # 'group: CTRL', 'group: ASD' indicates trait (ASD) status
age_row = 2  # 'age: 7', 'age: 9', etc. indicates age information
gender_row = None  # Gender information is not provided in the sample characteristics

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait data to binary format (0 for control, 1 for ASD)."""
    if value is None or ':' not in value:
        return None
    
    value = value.split(':', 1)[1].strip().upper()
    if value == 'CTRL' or value == 'CONTROL':
        return 0
    elif value == 'ASD':
        return 1
    return None

def convert_age(value):
    """Convert age data to continuous format."""
    if value is None or ':' not in value:
        return None
    
    value = value.split(':', 1)[1].strip()
    # Handle cases like "20/21" or "21/23" by averaging
    if '/' in value:
        ages = [int(age) for age in value.split('/')]
        return sum(ages) / len(ages)
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender data to binary format (0 for female, 1 for male)."""
    # This function is defined but not used as gender data is not available
    if value is None or ':' not in value:
        return None
    
    value = value.split(':', 1)[1].strip().lower()
    if value in ['female', 'f']:
        return 0
    elif value in ['male', 'm']:
        return 1
    return None

# 3. Save Metadata (Initial Filtering)
# Determine is_trait_available based on whether trait_row is None
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None)
if trait_row is not None:
    # Create a DataFrame with sample characteristics in a row-wise format
    data = []
    for key, values in sample_characteristics_dict.items():
        for value in values:
            data.append([value])
    clinical_data = pd.DataFrame(data)
    
    # Use geo_select_clinical_features to extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [0.0, nan]}
Clinical data saved to ../../output/preprocess/Autism_spectrum_disorder_(ASD)/clinical_data/GSE113842.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1415670_PM_at', '1415671_PM_at', '1415672_PM_at', '1415673_PM_at',
       '1415674_PM_a_at', '1415675_PM_at', '1415676_PM_a_at', '1415677_PM_at',
       '1415678_PM_at', '1415679_PM_at', '1415680_PM_at', '1415681_PM_at',
       '1415682_PM_at', '1415683_PM_at', '1415684_PM_at', '1415685_PM_at',
       '1415686_PM_at', '1415687_PM_a_at', '1415688_PM_at', '1415689_PM_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Affymetrix probeset IDs (format with "_at", "_s_at", "_x_at" suffix patterns)
# rather than standard human gene symbols like BRCA1, TP53, etc.
# These probesets would need to be mapped to gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1415670_PM_at', '1415671_PM_at', '1415672_PM_at', '1415673_PM_at', '1415674_PM_a_at'], 'GB_ACC': ['BC024686', 'NM_013477', 'NM_020585', 'NM_133900', 'NM_021789'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Mus musculus', 'Mus musculus', 'Mus musculus', 'Mus musculus', 'Mus musculus'], 'Annotation Date': ['Aug 10, 2010', 'Aug 10, 2010', 'Aug 10, 2010', 'Aug 10, 2010', 'Aug 10, 2010'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['GenBank', 'GenBank', 'GenBank', 'GenBank', 'GenBank'], 'Target Description': ['gb:BC024686.1 /DB_XREF=gi:19354080 /FEA=FLmRNA /CNT=416 /TID=Mm.26422.1 /TIER=FL+Stack /STK=110 /UG=Mm.26422 /LL=54161 /UG_GENE=Copg1 /DEF=Mus musculus, coatomer protein complex, subunit gamma 1, clone MGC:30335 IMAGE:3992144, mRNA, complete cds. /PROD=coatomer protein complex, subunit gamma 1 /FL=gb:AF187079.1 gb:BC024686.1

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in the annotation data that contain probe IDs and gene symbols
# Looking at the gene_annotation preview, I can see:
# - 'ID' column contains probe IDs (like '1415670_PM_at')
# - 'Gene Symbol' column contains the corresponding gene symbols (like 'Copg')

# 2. Extract these two columns from the gene annotation dataframe to create the mapping
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')

# 3. Apply the gene mapping to convert probe-level measurements to gene expression
# This function will split expression values for probes that map to multiple genes
# and then sum values for each gene from all contributing probes
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the first few rows of the mapped gene data
print("Gene data after mapping to gene symbols:")
print(gene_data.head())

Gene data after mapping to gene symbols:
            GSM3120892  GSM3120893  GSM3120894  GSM3120895  GSM3120896  \
Gene                                                                     
A130033P14    4.039066    4.192930    4.128634    3.988778    3.336130   
A430075N02    4.264285    4.319175    4.591880    4.228277    3.879057   
A630043P06    3.667454    4.484825    4.563515    4.573879    3.933827   
A730034C02   12.692358   11.214867    9.627281    8.557741   15.380640   
A830091E24    3.606027    3.780010    4.225027    3.949768    3.420589   

            GSM3120897  GSM3120898  GSM3120899  GSM3120964  GSM3120965  ...  \
Gene                                                                    ...   
A130033P14    3.814148    4.979083    4.338055    4.626518    4.631608  ...   
A430075N02    3.999499    4.272288    3.856207    4.751322    4.695923  ...   
A630043P06    4.060068    4.100830    4.268562    6.168188    6.319480  ...   
A730034C02   12.898844   10.628091   11.30431